# 🕷️ Scraping Automático de Tablas INE

## 📋 Descripción del Proyecto

Este notebook implementa una solución **automática y estandarizada** para hacer web scraping de tablas paginadas del Instituto Nacional de Estadísticas (INE) de Chile.

### ✨ Características principales:
- ✅ **Detección automática** del número de páginas
- ✅ **Procesamiento inteligente** de tablas por ID
- ✅ **Exportación organizada** (archivos individuales + combinado)
- ✅ **Configuración flexible** para diferentes URLs
- ✅ **Manejo robusto de errores**

### 📁 Estructura de archivos generados:
- `{prefix}_page1.csv`, `{prefix}_page2.csv`, etc. (páginas individuales)
- `{prefix}_combined.csv` (todas las páginas combinadas)

---

## 📦 Librerías y Configuración Inicial

Importamos todas las librerías necesarias para el web scraping:

- **`pandas`**: Manipulación y análisis de datos
- **`selenium`**: Automatización del navegador web
- **`webdriver_manager`**: Gestión automática del ChromeDriver
- **`time`**: Control de esperas y pausas

In [1]:
# 📌 Setup inicial y librerías
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 🚗 Función: Inicialización del Driver

Esta función configura y inicializa el navegador Chrome con las opciones optimizadas para scraping:

- **Modo headless**: Opción para ejecutar sin interfaz gráfica
- **Configuraciones de seguridad**: `--no-sandbox` y `--disable-dev-shm-usage`
- **ChromeDriver automático**: Descarga y gestiona la versión correcta

In [2]:
# 📌 1. Inicializar driver
def init_driver(headless: bool = True) -> webdriver.Chrome:
    """
    Inicializa el driver de Chrome con opciones configuradas.
    
    Args:
        headless (bool): Ejecutar en modo headless o no
        
    Returns:
        webdriver.Chrome: Instancia de ChromeDriver configurada
    """
    options = Options()
    if headless:
        options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver

print("✅ Función init_driver() definida")

✅ Función init_driver() definida


## 📊 Función: Extracción de Tabla por ID

Esta función localiza y extrae una tabla específica de la página usando su ID:

- **Espera inteligente**: Usa `WebDriverWait` para esperar que la tabla cargue
- **Extracción robusta**: Convierte HTML a DataFrame con `pandas.read_html()`
- **Debug automático**: Si falla, lista todos los IDs de tablas disponibles

In [3]:
# 📌 2. Función para scrapear tabla por ID
def scrape_table(driver, wait, table_id):
    """
    Extrae una tabla específica por su ID.
    
    Args:
        driver: Instancia del WebDriver
        wait: Objeto WebDriverWait para esperas
        table_id (str): ID de la tabla a extraer
        
    Returns:
        pd.DataFrame: Datos de la tabla extraída
    """
    try:
        table = wait.until(EC.presence_of_element_located((By.ID, table_id)))
        html = table.get_attribute('outerHTML')
        df = pd.read_html(html)[0]
        print(f"✅ Tabla extraída con ID '{table_id}'.")
        return df
    except Exception as e:
        print(f"❌ No se encontró la tabla con ID '{table_id}': {e}")
        # DEBUG: listar IDs de tablas disponibles
        tables = driver.find_elements(By.TAG_NAME, "table")
        ids = [t.get_attribute("id") for t in tables]
        print(f"🔎 IDs de tablas encontrados en la página: {ids}")
        return pd.DataFrame()

print("✅ Función scrape_table() definida")

✅ Función scrape_table() definida


## 📄 Funciones: Gestión de Paginación

Estas funciones manejan la navegación automática entre páginas:

### `get_total_pages()`:
- Detecta automáticamente cuántas páginas hay disponibles
- Busca elementos con clase `"PageNumber"`

### `go_to_page()`:
- Navega a una página específica
- Incluye validaciones y manejo de errores

In [4]:
# 📌 3. Función para detectar número total de páginas
def get_total_pages(driver, wait):
    """
    Detecta automáticamente el número total de páginas disponibles.
    
    Returns:
        int: Número total de páginas
    """
    try:
        # Buscar elementos de paginación
        pages = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "PageNumber")))
        total_pages = len(pages)
        print(f"🔍 Se detectaron {total_pages} páginas en total")
        return total_pages
    except Exception as e:
        print(f"⚠️ No se pudo detectar paginación: {e}")
        return 1  # Asume 1 página si no hay paginación

# 📌 4. Función para cambiar de página
def go_to_page(driver, wait, page_number):
    """
    Navega a una página específica.
    
    Args:
        page_number (int): Número de página (índice 0)
        
    Returns:
        bool: True si el cambio fue exitoso
    """
    try:
        pages = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "PageNumber")))
        if page_number < len(pages):
            pages[page_number].click()
            print(f"✅ Cambiado a la página {page_number + 1}")
            time.sleep(5)  # espera tras el cambio de página
            return True
        else:
            print(f"⚠️ Página {page_number + 1} no existe")
            return False
    except Exception as e:
        print(f"❌ Error al cambiar a la página {page_number + 1}: {e}")
        return False

print("✅ Funciones de paginación definidas")

✅ Funciones de paginación definidas


## 🚀 Función Principal: Scraping Automático

Esta es la función principal que orquesta todo el proceso de scraping:

### Características:
- ✅ **Automática**: Detecta y procesa todas las páginas
- ✅ **Flexible**: Configurable para diferentes URLs y tablas
- ✅ **Robusta**: Maneja errores y continúa el proceso
- ✅ **Organizada**: Genera archivos individuales y combinados

### Parámetros:
- `url`: URL objetivo
- `table_id`: ID de la tabla a extraer
- `output_prefix`: Prefijo para los archivos
- `headless`: Modo de ejecución del navegador

In [5]:
# 📌 5. Función principal de scraping automático
def scrape_all_pages_automatically(url, table_id, output_prefix="table", headless=True):
    """
    Función principal para hacer scraping automático de todas las páginas disponibles.
    
    Args:
        url (str): URL objetivo para el scraping
        table_id (str): ID de la tabla a extraer
        output_prefix (str): Prefijo para los archivos de salida
        headless (bool): Ejecutar en modo headless o no
    
    Returns:
        pd.DataFrame: DataFrame combinado con todos los datos
    """
    print(f"🚀 Iniciando scraping automático de: {url}")
    print(f"🎯 Buscando tabla con ID: {table_id}")
    
    # Inicializar driver
    driver = init_driver(headless=headless)
    wait = WebDriverWait(driver, 40)
    
    try:
        # Navegar a la URL
        driver.get(url)
        time.sleep(5)
        
        # Detectar número total de páginas
        total_pages = get_total_pages(driver, wait)
        
        # Lista para almacenar todos los DataFrames
        all_dataframes = []
        
        # Iterar por todas las páginas
        for page_num in range(total_pages):
            print(f"\n📄 Procesando página {page_num + 1} de {total_pages}...")
            
            # Si no es la primera página, navegar a la página específica
            if page_num > 0:
                success = go_to_page(driver, wait, page_num)
                if not success:
                    print(f"⚠️ Saltando página {page_num + 1}")
                    continue
            
            # Extraer tabla de la página actual
            df = scrape_table(driver, wait, table_id)
            
            if not df.empty:
                # Guardar página individual
                output_path = f"../data/raw/{output_prefix}_page{page_num + 1}.csv"
                df.to_csv(output_path, index=False, encoding='utf-8-sig')
                print(f"✅ Página {page_num + 1} exportada: {len(df)} filas")
                
                # Añadir a la lista para combinación
                all_dataframes.append(df)
            else:
                print(f"⚠️ Página {page_num + 1} está vacía")
        
        # Combinar todas las páginas
        if all_dataframes:
            df_combined = pd.concat(all_dataframes, ignore_index=True)
            combined_path = f"../data/raw/{output_prefix}_combined.csv"
            df_combined.to_csv(combined_path, index=False, encoding='utf-8-sig')
            
            print(f"\n🎉 SCRAPING COMPLETADO:")
            print(f"   📊 Total de páginas procesadas: {len(all_dataframes)}")
            print(f"   📊 Total de filas extraídas: {len(df_combined)}")
            print(f"   📁 Archivo combinado: {combined_path}")
            
            return df_combined
        else:
            print("❌ No se pudieron extraer datos de ninguna página")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error durante el scraping: {e}")
        return pd.DataFrame()
    
    finally:
        driver.quit()
        print("🏁 Driver cerrado")

print("✅ Función principal scrape_all_pages_automatically() definida")

✅ Función principal scrape_all_pages_automatically() definida


## ⚙️ Configuración y Ejecución

Aquí configuramos los parámetros para nuestro scraping automático:

### 🔧 Parámetros configurables:

| Parámetro | Descripción | Valor Actual |
|-----------|-------------|--------------|
| `URL_TARGET` | URL objetivo del scraping | INE Chile |
| `TABLE_ID` | ID de la tabla a extraer | `"tabletofreeze"` |
| `OUTPUT_PREFIX` | Prefijo de archivos | `"ine_table"` |
| `HEADLESS_MODE` | Modo sin interfaz gráfica | `False` (visible) |

> **💡 Tip**: Cambia `HEADLESS_MODE = True` para ejecutar en modo silencioso.

In [6]:
# 📌 6. Configuración Principal - CAMBIA ESTOS VALORES SEGÚN TU NECESIDAD

# 🔧 URL objetivo para el scraping
URL_TARGET = "https://stat.ine.cl/Index.aspx?lang=es&SubSessionId=78e0518e-d028-4bf8-8d80-444b7277907c"

# 🎯 ID de la tabla a extraer
TABLE_ID = "tabletofreeze"

# 📁 Prefijo para los archivos (se crearán: {prefix}_page1.csv, {prefix}_page2.csv, etc.)
OUTPUT_PREFIX = "ine_table"

# 👁️ Modo de ejecución (False = ver navegador, True = modo silencioso)
HEADLESS_MODE = False

print("✅ Configuración establecida:")
print(f"   🌐 URL: {URL_TARGET[:50]}...")
print(f"   📊 Tabla ID: {TABLE_ID}")
print(f"   📁 Prefijo: {OUTPUT_PREFIX}")
print(f"   👁️  Headless: {HEADLESS_MODE}")

✅ Configuración establecida:
   🌐 URL: https://stat.ine.cl/Index.aspx?lang=es&SubSessionI...
   📊 Tabla ID: tabletofreeze
   📁 Prefijo: ine_table
   👁️  Headless: False


## 🚀 Ejecutar Scraping Automático

¡Hora de poner en marcha nuestro scraper automático!

### Lo que va a suceder:
1. 🌐 Se abrirá el navegador y navegará a la URL
2. 🔍 Detectará automáticamente el número de páginas
3. 📄 Procesará cada página secuencialmente
4. 💾 Guardará archivos individuales por página
5. 🔗 Creará un archivo combinado con todos los datos
6. 📊 Mostrará un resumen final

> **⚠️ Importante**: Esta celda puede tomar varios minutos dependiendo del número de páginas y la velocidad de internet.

In [7]:
# 🚀 EJECUTAR SCRAPING AUTOMÁTICO
print("=" * 60)
print("🕷️  INICIANDO SCRAPING AUTOMÁTICO")
print("=" * 60)

# Ejecutar la función principal
result_df = scrape_all_pages_automatically(
    url=URL_TARGET,
    table_id=TABLE_ID,
    output_prefix=OUTPUT_PREFIX,
    headless=HEADLESS_MODE
)

print("\n" + "=" * 60)
print("🏁 PROCESO COMPLETADO")
print("=" * 60)

🕷️  INICIANDO SCRAPING AUTOMÁTICO
🚀 Iniciando scraping automático de: https://stat.ine.cl/Index.aspx?lang=es&SubSessionId=78e0518e-d028-4bf8-8d80-444b7277907c
🎯 Buscando tabla con ID: tabletofreeze
⚠️ No se pudo detectar paginación: Message: 


📄 Procesando página 1 de 1...
✅ Tabla extraída con ID 'tabletofreeze'.
✅ Página 1 exportada: 81 filas

🎉 SCRAPING COMPLETADO:
   📊 Total de páginas procesadas: 1
   📊 Total de filas extraídas: 81
   📁 Archivo combinado: ../data/raw/ine_table_combined.csv
🏁 Driver cerrado

🏁 PROCESO COMPLETADO


/var/folders/88/qq1xyp3n6x3dtbhw0fqlgvrm0000gn/T/ipykernel_1834/2190260432.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


## 📊 Análisis de Resultados

Vamos a analizar los datos extraídos para verificar que el scraping fue exitoso:

### Lo que veremos:
- 📏 **Dimensiones**: Número de filas y columnas
- 🏷️ **Columnas**: Nombres de las columnas extraídas
- 📄 **Muestra**: Primeras filas de los datos
- 📈 **Estadísticas**: Información básica sobre el dataset

In [8]:
# 📊 ANÁLISIS DE RESULTADOS

if not result_df.empty:
    print("🎉 ¡SCRAPING EXITOSO!")
    print("\n📋 RESUMEN FINAL:")
    print(f"   🔢 Columnas: {list(result_df.columns)}")
    print(f"   📏 Dimensiones: {result_df.shape}")
    print(f"   📊 Total de filas: {len(result_df)}")
    print(f"   📊 Total de columnas: {len(result_df.columns)}")
    
    print(f"\n📄 PRIMERAS 5 FILAS:")
    display(result_df.head())
    
    print(f"\n📈 INFORMACIÓN DEL DATASET:")
    display(result_df.info())
    
    print(f"\n📋 ESTADÍSTICAS DESCRIPTIVAS:")
    try:
        display(result_df.describe())
    except:
        print("   ℹ️ No se pueden mostrar estadísticas (datos no numéricos)")
        
else:
    print("❌ NO SE OBTUVIERON DATOS")
    print("   🔧 Verifica la configuración:")
    print(f"      • URL: {URL_TARGET}")
    print(f"      • Table ID: {TABLE_ID}")
    print("   🌐 Verifica que la página esté disponible")
    print("   🔍 Revisa los mensajes de error anteriores")

🎉 ¡SCRAPING EXITOSO!

📋 RESUMEN FINAL:
   🔢 Columnas: [('Indicador', 'Sexo', 'Región', 'Región', 'Trimestre Móvil'), ('Indicador', 'Sexo', 'Región', 'Región', 'Unnamed: 1_level_4'), ('Tasa de ocupación en el sector informal (proyecciones base 2002)Tasa de ocupación en el sector informal (proyecciones base 2017)', 'Ambos sexosHombresMujeres', 'Total país', 'Unnamed: 2_level_3', 'Unnamed: 2_level_4'), ('Tasa de ocupación en el sector informal (proyecciones base 2002)Tasa de ocupación en el sector informal (proyecciones base 2017)', 'Ambos sexosHombresMujeres', 'Región de Arica y Parinacota', 'Unnamed: 3_level_3', 'Unnamed: 3_level_4'), ('Tasa de ocupación en el sector informal (proyecciones base 2002)Tasa de ocupación en el sector informal (proyecciones base 2017)', 'Ambos sexosHombresMujeres', 'Región de Tarapacá', 'Unnamed: 4_level_3', 'Unnamed: 4_level_4'), ('Tasa de ocupación en el sector informal (proyecciones base 2002)Tasa de ocupación en el sector informal (proyecciones base 2017

Indicador                     \
             Sexo                      
           Región                      
           Región                      
  Trimestre Móvil Unnamed: 1_level_4   
0    2017 jul-sep                NaN   
1    2017 ago-oct                NaN   
2    2017 sep-nov                NaN   
3    2017 oct-dic                NaN   
4    2017 nov-ene                NaN   

  Tasa de ocupación en el sector informal (proyecciones base 2002)Tasa de ocupación en el sector informal (proyecciones base 2017)  \
                                                                                                         Ambos sexosHombresMujeres   
                                                                                                                        Total país   
                                                                                                                Unnamed: 2_level_3   
                                                                                                                Unnamed: 2_level_4   
0                                               16.0                                                                                 
1                                               16.1                                                                                 
2                                               16.0                                                                                 
3                                               15.9                                                                                 
4                                               15.5                                                                                 

                                                                         \
                                                                          
  Región de Arica y Parinacota Región de Tarapacá Región de Antofagasta   
            Unnamed: 3_level_3 Unnamed: 4_level_3    Unnamed: 5_level_3   
            Unnamed: 3_level_4 Unnamed: 4_level_4    Unnamed: 5_level_4   
0                         21.0               21.9                  13.2   
1                         20.4               22.2                  12.8   
2                         19.2               20.5                  11.9   
3                         19.5               19.9                  12.0   
4                         19.2               19.5                  12.7   

                                                              \
                                                               
   Región de Atacama Región de Coquimbo Región de Valparaíso   
  Unnamed: 6_level_3 Unnamed: 7_level_3   Unnamed: 8_level_3   
  Unnamed: 6_level_4 Unnamed: 7_level_4   Unnamed: 8_level_4   
0               17.5               21.3                 14.6   
1               19.7               21.5                 14.4   
2               19.9               21.1                 14.8   
3               18.3               21.1                 14.2   
4               15.3               20.5                 13.4   

                                    \
                                     
  Región Metropolitana de Santiago   
                Unnamed: 9_level_3   
                Unnamed: 9_level_4   
0                             14.5   
1                             14.6   
2                             14.5   
3                             14.7   
4                             14.4   

                                                                      \
                                                                       
  Región del Libertador Gral. Bernardo O'Higgins    Región del Maule   
                             Unnamed: 10_level_3 Unnamed: 11_level_3   
                             Unnamed: 10_level_4 Unnamed: 11_level_4   
0                                           13.9                14.9   
1                                           13.0             


📈 INFORMACIÓN DEL DATASET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 19 columns):
 #   Column                                                                                                                                                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                      --------------  -----  
 0   (Indicador, Sexo, Región, Región, Trimestre Móvil)                                                                                                                                                                                                          81 non-null     object 
 1   (Indicador, Sexo, Reg

None


📋 ESTADÍSTICAS DESCRIPTIVAS:


Indicador  \
                    Sexo   
                  Región   
                  Región   
      Unnamed: 1_level_4   
count                0.0   
mean                 NaN   
std                  NaN   
min                  NaN   
25%                  NaN   
50%                  NaN   
75%                  NaN   
max                  NaN   

      Tasa de ocupación en el sector informal (proyecciones base 2002)Tasa de ocupación en el sector informal (proyecciones base 2017)  \
                                                                                                             Ambos sexosHombresMujeres   
                                                                                                                            Total país   
                                                                                                                    Unnamed: 2_level_3   
                                                                                                                    Unnamed: 2_level_4   
count                                          81.000000                                                                                 
mean                                           16.104938                                                                                 
std                                             1.027363                                                                                 
min                                            12.700000                                                                                 
25%                                            15.800000                                                                                 
50%                                            16.100000                                                                                 
75%                                            16.600000                                                                                 
max                                            17.800000                                                                                 

                                                                             \
                                                                              
      Región de Arica y Parinacota Región de Tarapacá Región de Antofagasta   
                Unnamed: 3_level_3 Unnamed: 4_level_3    Unnamed: 5_level_3   
                Unnamed: 3_level_4 Unnamed: 4_level_4    Unnamed: 5_level_4   
count                    81.000000          81.000000             81.000000   
mean                     21.067901          21.560494             13.140741   
std                       2.747491           2.372798              1.609253   
min                      12.300000          14.000000              9.600000   
25%                      19.500000          20.600000             11.900000   
50%                      20.900000          22.300000             13.000000   
75%                      22.900000          23.200000             14.400000   
max                      26.700000          25.700000             17.300000   

                                                                  \
                                                                   
       Región de Atacama Región de Coquimbo Región de Valparaíso   
      Unnamed: 6_level_3 Unnamed: 7_level_3   Unnamed: 8_level_3   
      Unnamed: 6_level_4 Unnamed: 7_level_4   Unnamed: 8_level_4   
count          81.000000          81.000000            81.000000   
mean           16.407407          18.870370            15.904938   
std             1.543598           1.440264             1.775592   
min            12.700000          14.800000            11.600000   
25%            15.400000          18.000000            14.500000   
50%            16.500000          18.900000            16.200000   
75%            17.500000          19.900000            17.500000   
max        

## 🎯 Conclusiones y Próximos Pasos

### ✅ Lo que hemos logrado:
1. **Scraping automático** de tablas paginadas
2. **Detección inteligente** del número de páginas
3. **Extracción robusta** con manejo de errores
4. **Exportación organizada** en múltiples formatos

### 📁 Archivos generados:
- `ine_table_page1.csv`, `ine_table_page2.csv`, etc. (páginas individuales)
- `ine_table_combined.csv` (dataset completo)

### 🔧 Para usar con otros sitios web:
1. Cambia `URL_TARGET` por la nueva URL
2. Inspecciona la página para encontrar el `TABLE_ID` correcto
3. Ajusta `OUTPUT_PREFIX` para identificar tus archivos
4. Ejecuta el notebook

### 🚀 Posibles mejoras:
- Añadir filtros de datos
- Implementar limpieza automática
- Agregar visualizaciones
- Programar ejecuciones automáticas

---
**✨ ¡Felicidades! Has completado el scraping automático de tablas INE.**